In [1]:
""" Imports """
import re
from nltk.tokenize import word_tokenize, sent_tokenize
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

"""Global definitons"""
_start = 'S_START'
_end = 'S_END'

In [22]:
class WordItem:
    def __init__(self,word,count=0):
        self.word = word
        self.count = count

In [55]:
""" Word preprocessing """
def dataset(_fi='/home/jazzycrazzy/PythonScripts/dataset.csv', _fo = 'testfile.txt'):
    file_in = open(_fi)
    #file_out = open(_fo,'wb')

    words = [] #stores unique words encountered in the document as WordItem objects
    _dict = {} #temporary dictionary to maintain count of each word
    
    _dict['UNK'] = 0

    for l in file_in:
        #file_out.write(l+'\n')
        l = _start+' '+l+' '+_end
        split = word_tokenize(l.decode('utf-8'))
        for w in split:
            if len(w)==0:
                continue
            elif len(w) > 15: #if word's length is greater than 15 counting it as unknown
                _dict['UNK'] += 1
                continue
            if w not in _dict:
                _dict[w] = 1
            _dict[w] += 1
            
    _vocab = {} #dictionary with words as keys and values as indices of them in 'word' list
    _vocab['UNK'] = len(words)
    words.append(WordItem('UNK',_dict['UNK']))
    for k,v in _dict.iteritems():
        #if v > 9 and k != 'UNK':
        if k != 'UNK':
            _vocab[k] = len(words)
            words.append(WordItem(k,v))
        else:
            words[0].count += 1
    
    #cleaning up unnecessary memory
    del _dict
    file_in.close()
    #file_out.close()
    
    return _vocab, words

def UnigramTable(_vocab, words):
    """ Calculates probabilities based on count of each word present"""
    pow = 0.75
    totalFreqPow = 0.0
    unigramTable = {}
    
    l = [words[i].count**pow for i in range(len(_vocab))]
    totalFreqPow = np.sum(l)
    
    for i in range(len(_vocab)):
        unigramTable[i] = (words[i].count**pow)/totalFreqPow
    
    del l
    return unigramTable

def hotVector(wordIndex,vocabSize):
    """ Returns hot vector representation of a word """
    hVector = np.zeros(vocabSize)
    hVector[wordIndex-1] = 1
    return hVector

def softmax(net):
    """ calculates softmax score - target score normalized with noise scores and calculated as probability"""
    _exp = np.exp(net)
    return _exp/np.sum(_exp)

def sigmoid(net):
    """ Applies sigmoid logistic function on net """
    return 1.0/(1+np.exp(-net))

def randomIdx(k, vocabSize, current):
    """ Returns k indices from with unigram table randomly with respect to each word's probablity """
    global _unigramTable
    idxs = list(np.random.choice(vocabSize, k+1, False, p = _unigramTable.values()))
    if current in idxs:
        idxs.remove(current)
    else:
        del idxs[-1]
    return idxs
    
def softmaxCostGradient(net, target):
    prob = softmax(net)
    print(prob)
    
    
def negSamplingCostGradient(out, context, emb, vocabSize, learningRate, W_Output, k = 10):
    
    errorHidden = np.zeros(shape=(emb.size,1))
    
    actOut = sigmoid(out[context])
    negSamples = randomIdx(k, vocabSize, context)
    _negSamples = [-out[sample] for sample in negSamples]
    
    # error for context word
    e = -np.log(actOut) - np.sum(np.log(sigmoid(np.array(_negSamples))))
    
    """ calculating gradients for output vectors for both target and negative samples
    calculating hidden layer error for each context word """
    # Updating output weight vector for context word
    delta = actOut - 1
    errorHidden += delta * W_Output[:,context:context+1]
    W_Output[:,context:context+1] -= learningRate * np.reshape(delta * emb,(emb.size,1))
    
    # Updating output weight vectors for negative sampling
    for sample in negSamples:
        delta = sigmoid(out[sample])
        errorHidden += delta * W_Output[:,sample:sample+1]
        W_Output[:,sample:sample+1] -= learningRate * np.reshape(delta * emb,(emb.size,1))
    
    return errorHidden,e    
    
def skipgram(target,contextWords, vocabSize, learningRate, W_Embedding, W_Output):
    
    """
    will be called on each window with
    target: Target word index
    contextWords: Arrray of integers representing context words
    """
    loss = 0
    k = 10 #Number of negative samples
    emb = W_Embedding[target]
    out = np.matmul(emb,W_Output) # [1 x EmbSize].[EmbSize x VocabSize]
    #print out.shape
    _predicted = []
    EH = np.zeros(shape=(emb.size,1))
    for context in contextWords:
        #predicted = hotVector(context, vocabSize)
        #softmaxCostGradient(out,context)
        _EH,_e = negSamplingCostGradient(out, context, emb, vocabSize, learningRate, W_Output, k)
        EH += _EH
        loss += _e
        #EH += sof
        
    #updating hidden layer input vector embedding
    W_Embedding[target] -= learningRate * EH.T[0]
    return loss


In [56]:

""" Creates word embeddings in vector space representation """

""" Feedforward Neural Net Language model """
#Input layer

#Projection layer

#Hidden layer

#Output layer

#Initialization
fin='/home/jazzycrazzy/MTData/English/English-small.txt'#/home/jazzycrazzy/PythonScripts/dataset.csv'
fout = 'testfile.txt'
_vocab, words = dataset(fin, fout)
_unigramTable = UnigramTable(_vocab, words)

learningRate = 0.1
vocabSize = len(words)
emb_size = 10
win_size = 2
target = None
epoch = 20

print _vocab


# No need of hidden layer since when the embedding matrix is multiplied with hot vector 
#it essentially gives that embedding row
W_Embedding = np.random.randn(vocabSize,emb_size) #Embedding matrix
W_Output = np.random.randn(emb_size,vocabSize) #Outputlayer weight matrix Emb_size x Vocab

for _ in np.arange(epoch):
    
    totalLoss = 0
    loss = 0
    
    fileIn = open(fin)
    for l in fileIn:
        l = _start+' '+l+' '+_end
        tokens = word_tokenize(l.decode('utf-8'))
        #print 'tokens',tokens
        for token in tokens:
            
            loss = 0
            contextWords = []
            
            if token in _vocab:
                target = _vocab[token]
                trgtIdx = tokens.index(token)
                cntxtIdxs = range(trgtIdx-win_size, trgtIdx+win_size+1)
                cntxtIdxs.remove(trgtIdx)
                for idx in cntxtIdxs:
                    #check for first word and last word and use UNK for context words for window where words not available
                    if idx >-1 and idx < len(tokens) and tokens[idx] in _vocab:
                        contextWords = np.append(contextWords, _vocab[tokens[idx]])
                    else:
                        contextWords = np.append(contextWords, _vocab['UNK'])
                loss += skipgram(target, contextWords, vocabSize, learningRate, W_Embedding, W_Output)
        totalLoss += loss
    print 'Total Loss:',totalLoss
                

print(W_Embedding)

{u'enjoy': 1, u'S_END': 30, u'want': 3, u'tired': 4, u'ran': 5, u'is': 6, u'am': 7, u'see': 9, u'at': 10, u'have': 11, u'go': 12, u'tomorrow': 20, u'birth': 40, u'speak': 15, u'what': 17, u'how': 22, u'sun': 19, u'friends': 42, u'day': 43, u'graduate': 13, u'write': 16, u'to': 18, u'of': 52, u'enjoys': 24, u'has': 26, u'beach': 27, u'today': 28, u'?': 51, u'dad': 2, u'be': 31, u'we': 32, u'good': 33, u'read': 8, u'student': 37, u'sunny': 36, u'here': 38, u'every': 39, u'mom': 21, u'date': 41, 'UNK': 0, u'come': 23, u'you': 25, u'I': 34, u'a': 44, u'boy': 45, u'store': 46, u'your': 14, u'name': 47, u'did': 48, u'S_START': 49, u'work': 50, u'can': 29, u'night': 35, u'the': 53, u'nice': 54, u'where': 55, u'are': 56}


/home/jazzycrazzy/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:92: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/jazzycrazzy/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:103: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/jazzycrazzy/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:104: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Total Loss: 346.691826545
Total Loss: 178.768376317
Total Loss: 183.290073389
Total Loss: 166.625463328
Total Loss: 174.908476869
Total Loss: 169.296636938
Total Loss: 170.168831662
Total Loss: 177.859704209
Total Loss: 172.712286226
Total Loss: 176.027312374
Total Loss: 174.975126312
Total Loss: 176.584253707
Total Loss: 179.330062731
Total Loss: 178.340631869
Total Loss: 168.612763771
Total Loss: 169.049138441
Total Loss: 179.039235471
Total Loss: 172.019309351
Total Loss: 180.076219617
Total Loss: 169.766303609
[[ -3.13317690e-02  -6.98069634e-01   1.58624384e-01   6.50130627e-01
   -5.34281703e-01  -3.68388641e-01   1.12118685e+00   2.34374903e-01
    1.28035319e+00  -1.69041001e-02]
 [  2.21583361e+00  -2.60910986e-01   1.84376686e+00   2.14441012e+00
   -5.56048870e-01   6.13942590e-02  -1.30338472e-01  -7.99465715e-01
    5.05656544e-01  -8.99497493e-01]
 [  6.14369935e-01   1.32862548e+00   9.84956556e-01  -4.00023679e-01
   -1.12321765e+00   1.07646048e+00  -1.63702154e+00  -9

In [38]:
"""print _unigramTable
print words[0].word,words[0].count
print _vocab.values()[:10]
print _vocab.keys()[:10]
print words[_vocab.get('UNK')].count

print _vocab
#print W_Embedding
fig = plt.figure()
plt.scatter(W_Embedding[:,0:1], W_Embedding[:,1:2], W_Embedding[:,2:3])
plt.show()"""
y='La démarche terroriste est désormais une aberration qui ne séduit guère plus qu’une minorité.'
x = word_tokenize(_start+' '+y+' ')
net=np.array([-5, 2, 4, 3])
print 1/(1+np.exp(-net))
print sigmoid(net)
print np.negative(net)
print np.sum(np.log(sigmoid(np.negative(net))))

[ 0.00669285  0.88079708  0.98201379  0.95257413]
[ 0.00669285  0.88079708  0.98201379  0.95257413]
[ 5 -2 -4 -3]
-9.20038063902
